In [1]:
import os

In [4]:
%pwd

'c:\\DataScience\\Projects\\folder\\AIDRP-AI_Driven_Diabetes_Readmission_Prevention'

In [3]:
os.chdir('../')

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [13]:
from src.AIDRP.constants import *
from src.AIDRP.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.CatBoostClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [17]:
from sklearn.metrics import f1_score
import pandas as pd
import joblib

class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig) :
        self.config = config

    def eval_metrics(self,actual,pred):
        f1 = f1_score(actual,pred,average='micro')
        return f1
    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        predicted_values = model.predict(X_test)

        f1 = self.eval_metrics(y_test,predicted_values)

        # Saving metrics as local
        score = {"F1 Score" : f1}
        save_json(path=Path(self.config.metric_file_name), data=score)

In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-02-12 21:12:18,266 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-02-12 21:12:18,274 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-02-12 21:12:18,290 : INFO : common : yaml file: schema.yaml loaded successfully]
[2024-02-12 21:12:18,293 : INFO : common : created directory at: artifacts]
[2024-02-12 21:12:18,293 : INFO : common : created directory at: artifacts/model_evaluation]
[2024-02-12 21:12:18,588 : INFO : common : json file saved at: artifacts\model_evaluation\metrics.json]
